In [ ]:
%%html
<style>
.output_png {
    text-align:center;
    display:table-cell;
    vertical-align:middle;
}
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
    color: black !important;
    margin-left: auto;
    margin-right: auto;
}
div.output_area .rendered_html table{
    margin-left: auto;
    margin-right: auto;
}
</style>

In [ ]:
# Imports
import ipywidgets as widgets
from IPython.display import Javascript, display
import os
import matplotlib.pyplot as plt
import cv2
import aganitha_hocr
from aganitha_hocr import controller
import pandas as pd
from IPython.display import display
from IPython.display import Image
from IPython.display import clear_output
from ipywidgets import Button, HBox, VBox
from IPython.display import clear_output, FileLink, FileLinks


## Introduction
---


This demo takes in files like remittance advices, cash cheques and envelopes and extracts transactions from remittance advices. Finally the data is dumped into disk or pushed into a database.

## Process Workflow
---

<div style="text-align:center"><img src="./AR-Automation-process-flow.png" alt="WorkFlow"></div>
<div style="text-align:center"><em>AR-Automation-process-flow</em></div>

In [ ]:
import os, fnmatch
def find(pattern, path):
    result = []
    for root, dirs, files in os.walk(path):
        for name in files:
            if fnmatch.fnmatch(name, pattern):
                result.append(os.path.join(root, name))
    return result

def run_all(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))
    

In [ ]:
date_folders = os.listdir("/Users/adarsh/work/ar-automation/new_data/date-wise-data-complete/")
dd_date = widgets.Dropdown(
    options=date_folders,
    description='Select Date:',
    disabled=False,
    layout=widgets.Layout(align_items='center',width='350px'),
    style={'description_width':'initial'}
)
cust_names = ['OMG', 'IPG', 'MMS', 'GroupM', 'Squared', 'Katz']

filepath = '/Users/adarsh/work/ar-automation/new_data/date-wise-data-complete/'
count = 0
for date in date_folders:
    filenames = os.listdir(os.path.join(filepath, date))
    for filename in filenames:
        if filename.endswith(".html"):
            try:
                # print("Path", os.path.join(filepath,date,filename))
                file, extracted_params, temp, image = controller.main(filepath=os.path.join(filepath,date,filename))
                store_path = '/Users/adarsh/work/ar-automation/test3'
                controller.extract_data(file, temp, extracted_params, store_path, image)
            except TypeError:
                count = count+1
# print(count)
            

In [ ]:
## Download Merged  Spreadsheet

In [ ]:
def find_dir(pattern, path):
    result = []
    for root, dirs, files in os.walk(path):
        for name in dirs:
            if fnmatch.fnmatch(name, pattern):
                result.append(os.path.join(root, name))
    return result

cust_names = ['OMG', 'IPG', 'MMS', 'GroupM', '22Squared', 'Katz']
for name in cust_names:
    dir_paths = find_dir(name,'/Users/adarsh/work/ar-automation/test3/')
    dir_paths = [path for path in dir_paths if 'csv' in path]
    print(dir_paths)
    print("CSV PATHS FOR ", name)
    path_list = []
    for path in dir_paths:
        name_of_csvs = os.listdir(path)
        path_list = path_list + [os.path.join(path,csv) for csv in name_of_csvs if csv.endswith(".csv")]
        combined_csv = pd.concat([pd.read_csv(os.path.join(path,csv)) for csv in name_of_csvs if csv.endswith(".csv")], ignore_index=True)
        combined_csv.to_csv(name + '.csv', index=False)

    combined_csv = pd.concat([pd.read_csv(path) for path in path_list])
    print("Combined --> Start")
    combined_csv.to_csv(name + '.csv', index=False)
    display(combined_csv)
    print("Combined --> End")

In [ ]:
import pyexcel as p
from pyexcel_io import get_data
from pyexcel.cookbook import merge_all_to_a_book
import glob


In [ ]:
merge_all_to_a_book(glob.glob("*.csv"), "output.xlsx")

In [ ]:
headers = ["Deposit Date","CMG Account Number", "Lockbox Number", "Payor", "Advertiser", 
           "Payor Account Number", "Payor Check Number",  "Check Amount", "Remittance Line Number",
           "Gross Amount", "Discount/Adj.", "Net Amount Paid", "Status",
           "Path to Image",
          ]
df = pd.DataFrame(index=range(1000), columns=headers)

In [ ]:
df.head()


In [ ]:
filenames = ['MMS.csv', 'IPG.csv', '22Squared.csv', 'GroupM.csv', 'OMG.csv']
df.head()

In [ ]:
MMS = pd.read_csv('MMS.csv')
IPG = pd.read_csv('IPG.csv')
Squared = pd.read_csv('22Squared.csv')
GroupM = pd.read_csv('GroupM.csv')
OMG = pd.read_csv('OMG.csv')
Katz = pd.read_csv('Katz.csv')

MMS["Null Col"] = ''
IPG["Null Col"] = ''
Squared["Null Col"] = ''
GroupM["Null Col"] = ''
OMG["Null Col"] = ''
Katz["Null Col"] = ''

In [ ]:
!ls

In [ ]:
date_series = pd.concat([MMS['DATE'],IPG['Date'], OMG['Date'], Squared['Date'], GroupM['Check date'], Katz['Check Date']], axis=0, ignore_index=True)
check_number_series = pd.concat([MMS['CHECK NUMBER'],IPG['Check Number'], OMG['Check Number'], Squared['Check Number'], GroupM['Check number'], Katz['Check Number']], axis=0,ignore_index=True)
invoice_date_series = pd.concat([MMS['Invoice Date'],IPG['Invoice Date'], OMG['Invoice Date'], Squared['Invoice Date'], GroupM['Null Col'], Katz['Null Col']], axis=0, ignore_index=True)
invoice_number_series = pd.concat([MMS['Invoice Number'],IPG['Invoice Number'], OMG['Invoice Number'], Squared['Invoice Number'], GroupM['Invoice number'], Katz['Stn-Invoice']], axis=0, ignore_index=True)
amount_series = pd.concat([MMS['Amount'],IPG['Net Amount'], OMG['Net Amount'], Squared['Net Less Discount'], GroupM['Net Amount'], Katz['Paid Amount']], axis=0, ignore_index=True)
cust_series = pd.concat([MMS['Customer'],IPG['Customer'], OMG['Customer'], Squared['Customer'], GroupM['Customer'], Katz['Customer']], axis=0, ignore_index=True)
path_series = pd.concat([MMS['Path to Image'],IPG['Path to Image'], OMG['Path to Image'], Squared['Path to Image'], GroupM['Path to Image'], Katz['Path to Image']], axis=0, ignore_index=True)
lockbox_series = pd.concat([MMS['LockBox'],IPG['LockBox'], OMG['LockBox'], Squared['LockBox'], GroupM['LockBox'], Katz['LockBox']], axis=0, ignore_index=True)
remittance_line_number_series = pd.concat([MMS['Remittance Line Number'],IPG['Remittance Line Number'], OMG['Remittance Line Number'], Squared['Remittance Line Number'], GroupM['Remittance Line Number'], Katz['Remittance Line Number']], axis=0, ignore_index=True)
check_amount_series = pd.concat([MMS['AMOUNT PAID'],IPG['Check Total'], OMG['Check Amount'], Squared['Net Less Discount Total'], GroupM['Check amount'], Katz['Total']], axis=0, ignore_index=True)
gross_amount_series = pd.concat([MMS['Null Col'],IPG['Null Col'], OMG['Gross Amount'], Squared['Gross Amount'], GroupM['Total amount'], Katz['Null Col']], axis=0, ignore_index=True)
discount_amount_series = pd.concat([MMS['Null Col'],IPG['Null Col'], OMG['Null Col'], Squared['Discount'], GroupM['Null Col'], Katz['Null Col']], axis=0, ignore_index=True)
period_series = pd.concat([MMS['Null Col'],IPG['Null Col'], OMG['Null Col'], Squared['Null Col'], GroupM['Period'], Katz['Null Col']], axis=0, ignore_index=True)


In [ ]:
date_series = date_series.to_frame().reset_index()
check_number_series = check_number_series.to_frame().reset_index()
invoice_date_series = invoice_date_series.to_frame().reset_index()
invoice_number_series = invoice_number_series.to_frame().reset_index()
amount_series = amount_series.to_frame().reset_index()
remittance_line_number_series = remittance_line_number_series.to_frame()
cust_series = cust_series.to_frame()
path_series = path_series.to_frame()
lockbox_series = lockbox_series.to_frame()
check_amount_series = check_amount_series.to_frame().reset_index()
gross_amount_series = gross_amount_series.to_frame().reset_index()
discount_amount_series = discount_amount_series.to_frame().reset_index()
period_series = period_series.to_frame().reset_index()

In [ ]:
cust_series

In [ ]:
check_number_series[0] = check_number_series[0].astype(str)
print(check_number_series[0])

In [ ]:
df['date'] = date_series[0]

In [ ]:
print(df)

In [ ]:
df['Deposit Date'] = date_series[0]
df['Payor Check Number'] = check_number_series[0]
df['Invoice Date'] = invoice_date_series[0]
df['Invoice Number'] = invoice_number_series[0]
df['Period'] = period_series[0]
df['Net Amount Paid'] = amount_series[0]
df['Lockbox Number'] = lockbox_series
df['Payor'] = cust_series
df['Status'] = '1'
df["Remittance Line Number"] = remittance_line_number_series
df["Check Amount"] = check_amount_series[0]
df["Gross Amount"] = gross_amount_series[0]
df["Discount/Adj."] = discount_amount_series[0]
df['Path to Image'] = path_series
df["CMG Account Number"] = ''
df["Advertiser"] = ''
df["Payor Account Number"] = ''

In [ ]:
df2 = df[0:1122]

In [ ]:
for i in range(0,len(df2)):
    print(df2['Path to Image'][i])
    df2['Path to Image'][i] = df2['Path to Image'][i].replace('/Users/adarsh/work/ar-automation/new_data/date-wise-data-complete', '/data/date-wise-data-complete')

In [ ]:
df2["Check Amount"] = df2["Check Amount"].astype(str)
df2["Gross Amount"] = df2["Gross Amount"].astype(str)
df2["Net Amount Paid"] = df2["Net Amount Paid"].astype(str)
df2["Lockbox Number"] = df2["Lockbox Number"].astype(str)
df['Payor Check Number'] = df['Payor Check Number'].astype(str)

In [ ]:
for i in range(0,len(df2)):
    if '-' in df2["Check Amount"][i]:
        print("Okay")
        df2["Check Amount"][i] = df2["Check Amount"][i].replace("-", "")
        
for i in range(0,len(df2)):
    if '-' in df2["Gross Amount"][i]:
        print("Okay")
        df2["Gross Amount"][i] = df2["Gross Amount"][i].replace("-", "")
        
for i in range(0,len(df2)):
    if '-' in df2["Net Amount Paid"][i]:
        print("Okay")
        df2["Net Amount Paid"][i] = df2["Net Amount Paid"][i].replace("-", "")

for i in range(0,len(df2)):
    if '.0' in df2["Lockbox Number"][i]:
        print("Okay")
        df2["Lockbox Number"][i] = df2["Lockbox Number"][i].replace(".0", "")

In [ ]:
df2.tail(20)

In [ ]:
df2["Net Amount Paid"] = df2["Net Amount Paid"].astype(float)
df_sum = df2.groupby(["Path to Image"], as_index = False).agg({"Net Amount Paid": ['sum']})

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

In [ ]:
display(df2[df2["Path to Image"] == '/data/date-wise-data-complete/09.30.20/09.30.20-lb809036-1-7-addl-doc-01.jpg'])

In [ ]:
count = 0
count_true = 0
count_false = 0
for i in range(0,len(df2)):
    try:
        interim = df_sum.loc[df_sum["Path to Image"] == df2["Path to Image"][i]]
        # 
        if round(interim["Net Amount Paid"]["sum"].item(), 2) == round(float(df2["Check Amount"][i]), 2):
            df2["Status"][i] = 1
            count_true = count_true + 1
        else:
            df2["Status"][i] = 0
            count_false = count_false + 1
            print(round(interim["Net Amount Paid"]["sum"].item(), 2), " ", float(df2["Check Amount"][i]), df2["Path to Image"][i])
    except ValueError:
        count = count+1
        df2["Status"][i] = 2
print(count_true)
print(count_false)
print(count)

In [ ]:
pd.options.display.max_columns = 999

In [ ]:
df2.to_csv("Combined_data_ver5.csv", index=False)

In [ ]:
display(df2.head(100))

In [ ]:
df_sum = df2.groupby(["Path to Image"], as_index = False).agg({"Status": ['count']})

In [ ]:
df_sum